In [1]:
#Miami, December 27th, 2019
#Renato Kogeyama

# Due bad performance, I am updating the script to more efficient coding

# This script generates a citation ouput with patent|backward citation|cumulated backward citation | year
# The output generates a citation record by year, considering direct and indirect citation


In [2]:
import pandas as pd
import re
import matplotlib.pyplot as plot

import sys
sys.path.append('/home/rkogeyam/scripts/')
from sampler import sampler


In [3]:
%matplotlib inline

In [4]:
fname = r'/home/rkogeyam/PATENT_CITATION/uspatentcitation.tsv'
dst = '/home/rkogeyam/PATENT_CITATION/cit_tree.csv'

In [5]:
%%time
df=pd.read_csv(fname, sep='\t')

# sample_size=100
# df=sampler(fname, sample_size, sep='\t')

CPU times: user 3min 43s, sys: 15.1 s, total: 3min 58s
Wall time: 3min 58s


In [6]:
df['patent_id']=df['patent_id'].astype(str)

In [7]:
df['citation_id']=df['citation_id'].astype(str)

In [8]:
df.dtypes

uuid           object
patent_id      object
citation_id    object
date           object
name           object
kind           object
country        object
category       object
sequence        int64
dtype: object

In [9]:
# drop rows for which column citation_id has a length smaller than 5
indexNames = df[df['citation_id'].map(len) < 5].index
df.drop(indexNames , inplace=True)

In [10]:
# drop rows for which column patent_id has a length smaller than 5
indexNames = df[df['patent_id'].map(len) < 5].index
df.drop(indexNames , inplace=True)

In [11]:
%%time
#stripping non desired characters but keeping the originals for later check
#could merge unwanted information
cleaning_patent=lambda x:re.sub('([^a-zA-Z0-9]+)', "", x)
df['citation_id']=df['citation_id'].apply(cleaning_patent)
df['patent_id']=df['patent_id'].apply(cleaning_patent)
# df['citation_id'] = df['citation_id'].str.extract('([a-zA-Z0-9]+)', expand=True)
# df['patent_id'] = df['patent_id'].str.extract('([a-zA-Z0-9]+)', expand=True)

CPU times: user 7min 50s, sys: 108 ms, total: 7min 51s
Wall time: 7min 51s


In [12]:
#stripping trailing white spaces
df['patent_id'] = df['patent_id'].str.strip()
df['citation_id'] = df['citation_id'].str.strip()

In [13]:
%%time
#to avoid problems when converting object to datetime format
#replacing day or month would not affect the output
df.date.replace({'-00':'-01'}, regex=True, inplace=True)
#replacing the year could be more problematic
#for now, I am just dropping


CPU times: user 1min 7s, sys: 8 ms, total: 1min 7s
Wall time: 1min 7s


In [14]:
%%time
df['date']=pd.to_datetime(df.date,format='%Y-%m-%d', errors='coerce') #attention to date format - original data is year-month-day
# df['date_format']=pd.to_datetime(df.date,format='%Y-%m-%d', errors='coerce')

CPU times: user 31.5 s, sys: 894 ms, total: 32.4 s
Wall time: 32.3 s


In [15]:
%%time
#first, i'll run only patents with some kind of citation
#as we have many zero citations in the dataset, this should decrease processing time

cited_patents=df.groupby(['citation_id']).count().iloc[:, 1] #Series, patent-level
# cited_patents=df.groupby(['citation_id']).count() 

CPU times: user 1min 27s, sys: 2.13 s, total: 1min 29s
Wall time: 1min 29s


In [16]:
%%time
cited_patents.dropna(0, inplace=True) #Series

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 17.2 µs


In [17]:
cited_patents.shape

(7049261,)

In [18]:
%%time
cited_patents=cited_patents.reset_index() #Dataframe

CPU times: user 3.76 s, sys: 172 ms, total: 3.93 s
Wall time: 3.93 s


In [19]:
%%time
cited_patents.rename(columns={'patent_id': 'back_citation'}, inplace=True)

CPU times: user 593 ms, sys: 1 ms, total: 594 ms
Wall time: 593 ms


In [20]:
%%time
#this is the trick calculation
#I match the total of citations received by the citing patent (identified by patent_id)
#In the next step I sum all citations received by the citing patents

df=df.merge(cited_patents, how='left', left_on='patent_id', right_on='citation_id')
df.rename(columns={'back_citation': 'parent_back_citation'}, inplace=True)

CPU times: user 2min 21s, sys: 4.99 s, total: 2min 26s
Wall time: 2min 25s


In [21]:
%%time
#NaN in citations means no citation
df.fillna(0, inplace=True)

CPU times: user 53.9 s, sys: 2.02 s, total: 55.9 s
Wall time: 55.8 s


In [22]:
%%time
#now I collapse df to become a patent level dataframe
#citation_id_x is the original cited patent
# patent_level_df=df.groupby('citation_id_x').sum().iloc[:,2]
patent_level_df=df.groupby('citation_id_x').sum()

CPU times: user 1min 16s, sys: 1.39 s, total: 1min 18s
Wall time: 1min 17s


In [23]:
# lastly, i merge the number of citations to patent_level_df
# patent_level has the information about the citing patent

outcome=cited_patents.merge(patent_level_df, how='outer', left_index=True, right_index=True)

In [24]:
outcome=outcome.loc[:,['citation_id','back_citation','citation_id_y','parent_back_citation']].set_index('citation_id')

In [25]:
outcome.describe()

/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/numpy/lib/function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,back_citation,citation_id_y,parent_back_citation
count,7.049261e+06,0.0,7.049261e+06
mean,1.297236e+01,NaN,2.194859e+02
std,3.053004e+01,NaN,1.498394e+03
min,1.000000e+00,NaN,0.000000e+00
25%,NaN,NaN,NaN
50%,NaN,NaN,NaN
75%,NaN,NaN,NaN
max,3.922000e+03,NaN,2.274790e+05


In [26]:
%%time
outcome.to_csv(dst)

CPU times: user 37.9 s, sys: 60 ms, total: 38 s
Wall time: 38.1 s
